In [1]:
import os
import re
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
dt=[]
lb=[]
def load_review_dataset():
    i=0
    neg_path = 'neg'
    pos_path = 'pos'
    dataset=[]
    labels=[]
    for f_path in [neg_path,pos_path]:
        for txt_pth in [os.path.join(f_path,p) for p in os.listdir(f_path)]:
            with open(txt_pth,'r',encoding="utf-8") as txt_file:
                txt = txt_file.read()
                dataset.append(txt)
#                 print(f_path)
                labels.append(f_path)
                i=i+1
#     labels = np.full((50000), 0); y[100:200] = 1
    return dataset,labels,i
# load_review_dataset()
dt,lb,a=load_review_dataset()
print(len(dt))
print('finish')

50000
finish


## Preprocessing data

In [2]:
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
from nltk.tokenize.treebank import TreebankWordDetokenizer
def preproc(dt):
    print(len(dt))
    dtt = []
    labelsTest = []
    for i in range(len(dt)):    
        tokens1 = word_tokenize(dt[i])
    #     tokenisasi -> memecah setiap kalimat menjadi kata
    #     print("Tokenize")
    #     print(tokens1)
    #     mengubah menjadi lowercase
        normalized_words1 = [w.lower() for w in tokens1]
    #     print("lower")
    #     print(normalized_words1)
    #     mengubah tanda baca menjadi string kosong
        table = str.maketrans('', '', string.punctuation)
        punc_removed1 = [w.translate(table) for w in normalized_words1]
    #     print("punc_removed1:\n", punc_removed1)
    #     mengambil token-token yang berupa huruf alpabet
        isalpha_words1 = [word for word in punc_removed1 if word.isalpha()]
    #     print("isalpha_words1:\n", isalpha_words1)
    #     menghilangkan stop words bahasa inggris
        stop_words = set(stopwords.words('english'))
        stopWords_removed1 = [w for w in isalpha_words1 if not w in stop_words]
    #     print("stopWords_removed1:\n", stopWords_removed1)
    #     mencari kata dasar
        lemmatized_words1 = [lemmatizer.lemmatize(w) for w in stopWords_removed1]
    #     print("lemmatized_words1:\n", lemmatized_words1)
    #     untokenize
        dt[i] = TreebankWordDetokenizer().detokenize(lemmatized_words1)
        dtt.append(dt[i])
#         labelsTest.append(i)
    #     print("Untoken")
    #     print(x)
    print(len(dtt))
    return dtt
dataTest = preproc(dt)
print('finish')


50000
50000
finish


# K-folds

In [3]:
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
dataTest = np.asarray(dataTest)
lb = np.asarray(lb)
kf = KFold(n_splits=5)
kf.get_n_splits(dataTest[0:25000])
j=0
dataTraining=[]
dataTesting=[]
labelTraining =[]
labelTesting=[]
# binTraining=[]
# binTesting=[]
l1 = np.full((50000), 0); l1[25000:50000] = 1
for train_index, test_index in kf.split(dataTest[0:25000]):
#     print("\n\n iterasi keee ",j)
#     print("\n TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = dataTest[train_index], dataTest[test_index]
    y_train, y_test = lb[train_index], lb[test_index]
#     bin_train,bin_test = l1[train_index],l1[test_index]
#     print("\n TRAIN:", train_index+1000, "TEST:", test_index+1000, type(X_train))
    X_train1, X_test1 = dataTest[train_index+25000], dataTest[test_index+25000]
    y_train1, y_test1 = lb[train_index+25000], lb[test_index+25000]
#     bin_train1,bin_test1 = l1[train_index+25000],l1[test_index+25000]
    
    x_tr=np.concatenate((X_train,X_train1))
    y_tr=np.concatenate((y_train,y_train1))
    x_te = np.concatenate((X_test,X_test1))
    y_te = np.concatenate((y_test,y_test1))
#     b_tr = np.concatenate((bin_train,bin_train1))
#     b_te = np.concatenate((bin_test,bin_test1))
#     binTraining.append(b_tr)
#     binTesting.append(b_te)
    dataTraining.append(x_tr)
    labelTraining.append(y_tr)
    dataTesting.append(x_te)
    labelTesting.append(y_te) 
    j=j+1

print('finish')
print(len(dataTesting))

finish
5


## Fitur

In [4]:
from sklearn.metrics import confusion_matrix
fiturTraining = []
fiturTesting = []
for ii in range(5):
#     print(ii)
#   count vector
    count_vect = CountVectorizer(ngram_range=(1,1))
    tfidf_transformer = TfidfTransformer()
#   tfidf fitur training
    X_train_counts = count_vect.fit_transform(dataTraining[ii])
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    fiturTraining.append(X_train_tfidf)
#   fitur testing
    X_new_counts = count_vect.transform(dataTesting[ii])
    X_new_tfidf = tfidf_transformer.transform(X_new_counts)
    fiturTesting.append(X_new_tfidf)
print('finish')

finish


In [5]:
# print(len(dataTesting[0][0]))
# print(fiturTraining[0].shape)
# print(fiturTesting[0].shape)

## SVM

In [6]:
from sklearn import svm
print("svm")
for ii in range(5):
    print(ii)
# classifier
    classifier = LinearSVC()
    print(ii)
    classifier.fit(fiturTraining[ii],labelTraining[ii])
    print("pred")
    y_pred_tr = classifier.predict(fiturTraining[ii])
    print(confusion_matrix(labelTraining[ii], y_pred_tr))
    print(classification_report(labelTraining[ii],y_pred_tr))
#   new Data
    y_pred_te = classifier.predict(fiturTesting[ii])
    print(confusion_matrix(labelTesting[ii], y_pred_te))
    print(classification_report(labelTesting[ii],y_pred_te))

svm
0
0
pred
[[19743   257]
 [  235 19765]]
              precision    recall  f1-score   support

         neg       0.99      0.99      0.99     20000
         pos       0.99      0.99      0.99     20000

    accuracy                           0.99     40000
   macro avg       0.99      0.99      0.99     40000
weighted avg       0.99      0.99      0.99     40000

[[4431  569]
 [ 779 4221]]
              precision    recall  f1-score   support

         neg       0.85      0.89      0.87      5000
         pos       0.88      0.84      0.86      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000

1
1
pred
[[19776   224]
 [  203 19797]]
              precision    recall  f1-score   support

         neg       0.99      0.99      0.99     20000
         pos       0.99      0.99      0.99     20000

    accuracy                           0.99     40000
   macro avg     

## KNN

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
for ii in range(5):
#     CLASSIFIER
    knn = KNeighborsClassifier(n_neighbors=15)
    knn.fit(fiturTraining[ii], labelTraining[ii])
    y_pred = knn.predict(fiturTraining[ii])
    print(confusion_matrix(labelTraining[ii], y_pred))
    print(classification_report(labelTraining[ii], y_pred))
#   Testing
    y_pred_te = knn.predict(fiturTesting[ii])
    print(confusion_matrix(labelTesting[ii], y_pred_te))
    print(classification_report(labelTesting[ii],y_pred_te))

[[16105  3895]
 [ 3051 16949]]
              precision    recall  f1-score   support

         neg       0.84      0.81      0.82     20000
         pos       0.81      0.85      0.83     20000

    accuracy                           0.83     40000
   macro avg       0.83      0.83      0.83     40000
weighted avg       0.83      0.83      0.83     40000

[[3698 1302]
 [2227 2773]]
              precision    recall  f1-score   support

         neg       0.62      0.74      0.68      5000
         pos       0.68      0.55      0.61      5000

    accuracy                           0.65     10000
   macro avg       0.65      0.65      0.64     10000
weighted avg       0.65      0.65      0.64     10000

[[16627  3373]
 [ 2745 17255]]
              precision    recall  f1-score   support

         neg       0.86      0.83      0.84     20000
         pos       0.84      0.86      0.85     20000

    accuracy                           0.85     40000
   macro avg       0.85      0.85      

## Bayes

In [6]:
from sklearn.naive_bayes import BernoulliNB
classifier = BernoulliNB()
for ii in range(5):
    print(ii)
# classifier
#     classifier = LinearSVC()
    classifier.fit(fiturTraining[ii],labelTraining[ii])
    y_pred_tr = classifier.predict(fiturTraining[ii])
    print(confusion_matrix(labelTraining[ii], y_pred_tr))
    print(classification_report(labelTraining[ii],y_pred_tr))
#   new Data
    y_pred_te = classifier.predict(fiturTesting[ii])
    print(confusion_matrix(labelTesting[ii], y_pred_te))
    print(classification_report(labelTesting[ii],y_pred_te))


0
[[18746  1254]
 [ 2375 17625]]
              precision    recall  f1-score   support

         neg       0.89      0.94      0.91     20000
         pos       0.93      0.88      0.91     20000

    accuracy                           0.91     40000
   macro avg       0.91      0.91      0.91     40000
weighted avg       0.91      0.91      0.91     40000

[[4427  573]
 [1081 3919]]
              precision    recall  f1-score   support

         neg       0.80      0.89      0.84      5000
         pos       0.87      0.78      0.83      5000

    accuracy                           0.83     10000
   macro avg       0.84      0.83      0.83     10000
weighted avg       0.84      0.83      0.83     10000

1
[[18784  1216]
 [ 2242 17758]]
              precision    recall  f1-score   support

         neg       0.89      0.94      0.92     20000
         pos       0.94      0.89      0.91     20000

    accuracy                           0.91     40000
   macro avg       0.91      0.91  